# 模型保存及加载
本示例教程演示如何在MNIST数据集进行模型保存和加载，在训练完神经网络模型的时候，我们往往需要保存模型参数，以便做预测的时候能够省略训练步骤，直接加载模型参数。除此之外，在日常训练工作中我们会遇到一些突发情况，导致训练过程主动或被动的中断；抑或由于模型过于庞大，训练一个模型需要花费几天的训练时间；面对以上情况，Paddle中提供了很好地保存模型和提取模型的方法，支持从上一次保存状态开始训练，只要我们随时保存训练过程中的模型状态，就不用从初始状态重新训练。
下面将基于MNIST模型讲解paddle如何保存及加载模型，并恢复训练，网络结构部分的讲解省略。

## 环境
本教程基于paddle-develop编写，如果您的环境不是本版本，请先安装paddle-develop版本。

In [8]:
import paddle
import paddle.fluid as fluid
import paddle.hapi as hapi
from paddle.nn import functional
from paddle.hapi.model import Model
from paddle.vision.datasets import MNIST
from paddle.metric import Accuracy
from paddle.nn import Conv2d,Pool2D,Linear
from paddle.vision import LeNet
from paddle.static import InputSpec

print(paddle.__version__)
paddle.disable_static()

0.0.0


## 数据集
手写数字的MNIST数据集，包含60,000个用于训练的示例和10,000个用于测试的示例。这些数字已经过尺寸标准化并位于图像中心，图像是固定大小(28x28像素)，其值为0到1。该数据集的官方地址为：http://yann.lecun.com/exdb/mnist/
本例中我们使用飞桨自带的paddle.dataset完成mnist数据集的加载。

In [9]:
train_dataset = MNIST(mode='train')
test_dataset = MNIST(mode='test')

## 模型搭建

In [10]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = paddle.nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=2)
        self.max_pool1 = Pool2D(pool_size=2, pool_type='max', pool_stride=2)
        self.conv2 = Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1)
        self.max_pool2 = Pool2D(pool_size=2, pool_type='max', pool_stride=2)
        self.linear1 = Linear(in_features=16*5*5, out_features=120)
        self.linear2 = Linear(in_features=120, out_features=84)
        self.linear3 = Linear(in_features=84, out_features=10)

    def forward(self, x):
        x = self.conv1(x)
        x = functional.relu(x)
        x = self.max_pool1(x)
        x = functional.relu(x)
        x = self.conv2(x)
        x = self.max_pool2(x)
        x = paddle.reshape(x, shape=[-1, 16*5*5])
        x = self.linear1(x)
        x = functional.relu(x)
        x = self.linear2(x)
        x = functional.relu(x)
        x = self.linear3(x)
        x = functional.softmax(x)
        return x

## 模型训练
通过`Model` 构建实例，快速完成模型训练

In [11]:
inputs = InputSpec([None, 784], 'float32', 'x')
labels = InputSpec([None, 10], 'float32', 'x')
model = hapi.Model(LeNet(), inputs, labels)

optim = paddle.optimizer.Adam(learning_rate=0.001, parameters=model.parameters())

model.prepare(
    optim,
    paddle.nn.loss.CrossEntropyLoss(),
    Accuracy(topk=(1, 2))
    )
model.fit(train_dataset,
        test_dataset,
        epochs=1,
        batch_size=64,
        save_dir='mnist_checkpoint')


Epoch 1/1
step  10/938 - loss: 2.1389 - acc_top1: 0.2828 - acc_top2: 0.4516 - 16ms/step
step  20/938 - loss: 1.9412 - acc_top1: 0.4047 - acc_top2: 0.5195 - 14ms/step
step  30/938 - loss: 1.8458 - acc_top1: 0.4714 - acc_top2: 0.5708 - 14ms/step
step  40/938 - loss: 1.7914 - acc_top1: 0.5195 - acc_top2: 0.6133 - 13ms/step
step  50/938 - loss: 1.8215 - acc_top1: 0.5637 - acc_top2: 0.6531 - 13ms/step
step  60/938 - loss: 1.6824 - acc_top1: 0.5885 - acc_top2: 0.6729 - 13ms/step
step  70/938 - loss: 1.7004 - acc_top1: 0.6136 - acc_top2: 0.6989 - 13ms/step
step  80/938 - loss: 1.5939 - acc_top1: 0.6410 - acc_top2: 0.7264 - 13ms/step
step  90/938 - loss: 1.5671 - acc_top1: 0.6681 - acc_top2: 0.7514 - 13ms/step
step 100/938 - loss: 1.5495 - acc_top1: 0.6920 - acc_top2: 0.7736 - 13ms/step
step 110/938 - loss: 1.5818 - acc_top1: 0.7105 - acc_top2: 0.7908 - 13ms/step
step 120/938 - loss: 1.5283 - acc_top1: 0.7258 - acc_top2: 0.8046 - 13ms/step
step 130/938 - loss: 1.5800 - acc_top1: 0.7397 - acc_t

step 110/157 - loss: 1.4631 - acc_top1: 0.9678 - acc_top2: 0.9874 - 4ms/step
step 120/157 - loss: 1.4806 - acc_top1: 0.9695 - acc_top2: 0.9882 - 4ms/step
step 130/157 - loss: 1.4823 - acc_top1: 0.9710 - acc_top2: 0.9888 - 4ms/step
step 140/157 - loss: 1.4612 - acc_top1: 0.9730 - acc_top2: 0.9896 - 4ms/step
step 150/157 - loss: 1.4668 - acc_top1: 0.9741 - acc_top2: 0.9900 - 4ms/step
step 157/157 - loss: 1.4613 - acc_top1: 0.9742 - acc_top2: 0.9901 - 4ms/step
Eval samples: 10000
save checkpoint at /Users/chenlong/online_repo/book/paddle2.0_docs/mnist_checkpoint/final


## 保存模型
####  静态图保存模型API：
* fluid.io.save_vars(executor, dirname, main_program=None, vars=None, predicate=None, filename=None)<br>
1）通过接口中的 vars 指定需要保存的变量列表。<br>
2）将一个已经存在的程序（Program）赋值给接口中的 main_program，然后这个程序中的所有变量都将被保存下来。
第一种保存方式的优先级要高于第二种。 
* fluid.io.save_params(executor, dirname, main_program=None, filename=None) <br>
通过接口中的 main_program 指定好程序（Program），该接口会将所指定程序中的全部参数（Parameter）过滤出来，并将它们保存到 dirname 指定的文件夹或 filename 指定的文件中。 
* fluid.io.save_persistables(executor, dirname, main_program=None, filename=None) <br>
通过接口中的 main_program 指定好程序（Program），该接口会将所指定程序中的全部持久性变量（persistable==True）过滤出来，并将它们保存到 dirname 指定的文件夹或 filename 指定的文件中。 
* fluid.io.save_inference_model(dirname, feeded_var_names, target_vars, executor, main_program=None, model_filename=None, params_filename=None, export_for_deployment=True, program_only=False) <br>
存储预测模型时，一般通过 fluid.io.save_inference_model 接口对默认的 fluid.Program 进行裁剪，只保留预测 predict_var 所需部分。 裁剪后的 program 会保存在指定路径 ./infer_model/__model__ 下，参数会保存到 ./infer_model 下的各个独立文件。

####  动态图保存模型API：
* paddle.fluid.dygraph.save_dygraph(state_dict, model_path) <br>
该接口将传入的参数或优化器的 dict 保存到磁盘上,会根据 state_dict 的内容，自动给 model_path 添加 .pdparams 或者 .pdopt 后缀， 生成 model_path + ".pdparams" 或者 model_path + ".pdopt" 文件，state_dict 是通过 Layer 的 state_dict() 方法得到的。详细使用方法请参考：https://www.paddlepaddle.org.cn/documentation/docs/zh/api_cn/dygraph_cn/save_dygraph_cn.html#save-dygraph 
* paddle.incubate.hapi.model.Model.fit(train_data, epochs, batch_size, save_dir, log_freq) <br>
在使用model.fit函数进行网络循环训练时，在save_dir参数中指定保存模型的路径，save_freq指定写入频率，即可同时实现模型的训练和保存。mode.fit()只能保存模型参数，不能保存优化器参数，每个epoch结束只会生成一个.pdparams文件。可以边训练边保存，每次epoch结束会实时生成一个.pdparams文件。 
* paddle.incubate.hapi.model.Model.save(path) <br>
model.save(path)方法可以保存网络参数和优化器参数，每个epoch会生成两种文件 0.pdparams,0.pdopt，分别存储了模型参数和优化器参数，但是只会在整个模型训练完成后才会生成参数文件，path的格式为'dirname/file_prefix' 或 'file_prefix'，其中dirname指定路径名称，file_prefix 指定参数文件的名称。

In [5]:
# 方法一：使用动态图 model.save()保存模型和优化器参数信息
model.save('mnist_checkpoint/test')

In [ ]:
# 方法二：训练过程中实时保存每个epoch的模型参数
model.fit(train_dataset,
        test_dataset,
        epochs=2,
        batch_size=64,
        save_dir='mnist_checkpoint'
        )

## 加载模型参数

当恢复训练状态时，需要加载模型数据，此时我们可以使用加载函数从存储模型状态和优化器状态的文件中载入模型参数和优化器参数，如果不需要恢复优化器，则不必使用优化器状态文件。

#### 静态图加载模型参数
* fluid.io.load_vars<br>
通过执行器（Executor）加载指定目录中的变量。加载变量的方式有两种：
1）通过接口中的 vars 指定需要加载的变量列表。
2）将一个已经存在的程序（Program）赋值给接口中的 main_program，然后这个程序中的所有变量都将被加载。
第一种加载方式的优先级要高于第二种。
* fluid.io.load_params<br>
该接口从 main_program 指定的程序中过滤出全部参数（Parameter），并试图从 dirname 指定的文件夹或 filename 指定的文件中加载这些参数。
* fluid.io.load_persistables<br>
该接口从 main_program 指定的程序中过滤出全部持久性变量（persistable==True），并试图从 dirname 指定的文件夹或 filename 指定的文件中加载这些持久性变量。
* fluid.io.load_inference_model<br>
存储预测模型时，一般通过 fluid.io.save_inference_model 接口对默认的 fluid.Program 进行裁剪，只保留预测 predict_var 所需部分。 裁剪后的 program 会保存在指定路径 ./infer_model/__model__ 下，参数会保存到 ./infer_model 下的各个独立文件。

#### 动态图加载模型参数
* paddle.fluid.dygraph.load_dygraph(model_path)<br>
该接口尝试从磁盘中加载参数或优化器的 dict，该接口会同时加载 model_path + ".pdparams" 和 model_path + ".pdopt" 中的内容。其中model_path参数保存state_dict的文件的前缀，该路径不应该包括后缀 .pdparams 或 .pdopt。此函数返回两个 dict ，即从文件中恢复的参数 dict 和优化器 dict。
* paddle.incubate.hapi.model.Model.load(self, path, skip_mismatch=False, reset_optimizer=False)<br>
从存储的模型和优化器的参数文件种加载。如果不需要恢复优化器，则不必恢复优化器参数文件。

In [ ]:
# 使用动态图 model.load()加载模型和优化器参数信息
model.load('mnist_checkpoint/test')

## 如何判断模型是否准确的恢复训练呢？

理想的恢复训练是模型状态回到训练中断的时刻，恢复训练之后的梯度更新走向是和恢复训练前的梯度走向完全相同的。基于此，我们可以通过恢复训练后的损失变化，判断上述方法是否能准确的恢复训练。即从epoch 0结束时保存的模型参数和优化器状态恢复训练，校验其后训练的损失变化（epoch 1）是否和不中断时的训练完全一致。

说明：

恢复训练有如下两个要点：

* 保存模型时同时保存模型参数和优化器参数

* 恢复参数时同时恢复模型参数和优化器参数。

In [ ]:
import paddle
import paddle.fluid as fluid
import paddle.hapi as hapi
from paddle.nn import functional
from paddle.hapi.model import Model
from paddle.vision.datasets import MNIST
from paddle.metric import Accuracy
from paddle.nn import Conv2d,Pool2D,Linear
from paddle.vision import LeNet
from paddle.static import InputSpec
#
#
train_dataset = MNIST(mode='train')
test_dataset = MNIST(mode='test')

paddle.disable_static()
params_path = "mnist_checkpoint/test"

inputs = InputSpec([None, 784], 'float32', 'x')
labels = InputSpec([None, 10], 'float32', 'x')
model = hapi.Model(LeNet(), inputs, labels)
optim = paddle.optimizer.Adam(learning_rate=0.001, parameters=model.parameters())
model.prepare( 
      optim,
      paddle.nn.loss.CrossEntropyLoss(),
      Accuracy(topk=(1, 2))
      )
model.load(params_path)
model.fit(train_data=train_dataset,
        eval_data=test_dataset,
        batch_size=64,
        epochs=5
        )

## 总结


以上就是用Mnist手写数字识别的例子对保存模型和加载模型进行讲解，Paddle提供很很多保存和加载的API方法，您可以根据自己的需求进行选择。